# Perform inference on your data with SkeletonDiffusion

If your data are in the same skeleton format as our trained model, you can perform inference from your data.
Give a sequence of keypoints representing the past, and run SkeletonDiffusion to predict future motions!

SkeletonDiffusion can run on the output of other models, for example methods for human pose estimation from images or video.
For an example, check out our demo on Huggingface.

## Select model and data type
Here we take as an example our model trained on AMASS, which follows the same parametrization (skeleton format) as SMPL.

In [1]:
# choose between 'amass' and 'amass-mano'
# model_dataset = 'amass' 
model_dataset = 'amass-mano'

checkpoint_path = f'./trained_models/hmp/{model_dataset}/diffusion/checkpoints/cvpr_release.pt'
num_samples = 50

## Load model's weights

In [2]:
import os
import torch
import numpy as np
import random

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

from src.eval_prepare_model import prepare_model, get_prediction, load_model_config_exp
from src.data import create_skeleton


def set_seed(seed=0):
    torch.use_deterministic_algorithms(True)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


In [3]:
set_seed(seed=0)

config, exp_folder = load_model_config_exp(checkpoint_path)
config['checkpoint_path'] = checkpoint_path
skeleton = create_skeleton(**config)   


model, device, *_ = prepare_model(config, skeleton, **config)

> GPU 0 ready: Quadro RTX 5000
> GPU 1 ready: Quadro P400
Loading Autoencoder checkpoint: ./trained_models/hmp/amass-mano/autoencoder/checkpoints/cvpr_release.pt ...
Diffusion is_ddim_sampling:  False
Loading Diffusion checkpoint: ./trained_models/hmp/amass-mano/diffusion/checkpoints/cvpr_release.pt ...


## Load given example or use your own

In [ ]:
# prepare input
# load input. Unit must be in meters
# obs sequence contains the hip or root joint, it has not been dropped yet. 
obs = np.load(f'figures/example_obs_{model_dataset}.npy') # (t_past, J, 3)

obs = torch.from_numpy(obs).to(device).float()
# obs = obs.unsqueeze(0) # remember to add batch size if not present

In [5]:
obs_in = skeleton.tranform_to_input_space(obs) 
pred = get_prediction(obs_in, model, num_samples=num_samples, **config) # [batch_size, n_samples, seq_length, num_joints, features]
pred = skeleton.transform_to_metric_space(pred)

In [6]:
# Proceed to your own task.
# For example, you can rank the output by the one with least limb stretching.
# Checkout other metrics in src.metrics 
# or the diversity ranking in metrics/utils.py (see example in other notebook)


from src.metrics.body_realism import limb_stretching_normed_mean


limbstretching = limb_stretching_normed_mean(pred, target=obs[..., 1:, :][0].unsqueeze(1), limbseq=skeleton.get_limbseq(), reduction='persample', obs_as_target=True)
limbstretching_sorted, indices =  torch.sort(limbstretching.squeeze(1), dim=-1, descending=False) 
